In [1]:
import sys
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F

import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

mnist_train = datasets.MNIST(root='./data', download=True, train=True, transform=ToTensor())
mnist_test  = datasets.MNIST(root='./data', download=True, train=False, transform=ToTensor())

train_dataloader = DataLoader(mnist_train, batch_size=32, shuffle=True)
test_dataloader  = DataLoader(mnist_test,  batch_size=32, shuffle=True)

model = nn.Sequential(
    nn.Linear(784, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(0, 10):
    loss_sum = 0
    for X, y in train_dataloader:
        X = X.reshape((-1, 784))
        y = F.one_hot(y, num_classes=10).type(torch.float32)

        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    print(loss_sum)

# Evaluate on the test set
model.eval()
with torch.no_grad():
    accurate = 0
    total = 0
    for X, y in test_dataloader:
        X = X.reshape((-1, 784))
        # (Keep y as integer class IDs for comparison)

        outputs = nn.functional.sigmoid(model(X))
        # Take predicted class with outputs.max(dim=1).indices
        y_pred = outputs.max(dim=1).indices
        correct_pred = (y == y_pred)

        total += correct_pred.size(0)
        accurate += correct_pred.type(torch.int).sum().item()

    print(accurate / total)

140.05450346227735
64.06041412614286
47.36411800957285
38.31910940725356
32.517161624506116
28.28913304361049
25.00705560366623
22.32012009707978
20.089054559357464
18.211595882748952
0.9765
